<a href="https://colab.research.google.com/github/akansha810/akansha810/blob/main/NLP_FakeNewsClassifierUsingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fake News Classifier Using LSTM

In [117]:
import pandas as pd
import numpy as np

In [118]:
df=pd.read_csv("/content/train.csv", error_bad_lines=False, engine="python")

<ipython-input-118-fbc243134916>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv("/content/train.csv", error_bad_lines=False, engine="python")
Skipping line 13535: field larger than field limit (131072)
Skipping line 19783: field larger than field limit (131072)
Skipping line 13541: Expected 5 fields in line 13541, saw 6
Skipping line 19782: Expected 5 fields in line 19782, saw 7
Skipping line 19783: Expected 5 fields in line 19783, saw 10
Skipping line 19785: Expected 5 fields in line 19785, saw 14
Skipping line 19787: Expected 5 fields in line 19787, saw 8
Skipping line 19789: Expected 5 fields in line 19789, saw 7
Skipping line 19790: Expected 5 fields in line 19790, saw 15
Skipping line 19791: Expected 5 fields in line 19791, saw 9
Skipping line 19792: Expected 5 fields in line 19792, saw 9
Skipping line 19793: Expected 5 fields in line 19793, saw 12
Skipping line 19794: E

In [119]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [120]:
df.isnull().values.any()

True

In [121]:
df.isnull().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

In [122]:
df.shape

(20822, 5)

In [123]:
## drop nan values
df=df.dropna()

In [124]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [125]:
df.shape

(18285, 5)

In [126]:
### Get the Independent features
X=df.drop("label",axis=1)

In [127]:
X

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20817,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20818,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20819,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20820,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [128]:
# get the dependent variable
y=df["label"]

In [129]:
import tensorflow as tf

In [130]:
from tensorflow.keras.layers import Embedding, LSTM , Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential

In [131]:
## Vocabulary size
voc_size=5000

One_Hot Representation

In [132]:
messages = X.copy()

In [133]:
messages["title"][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [134]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20817,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20818,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20819,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20820,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [135]:
#messages.reset_index(inplace=True)

In [136]:
import nltk
import re
from nltk.corpus import stopwords

In [137]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [145]:
### Preprocessing
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()
corpus=[]
for i in range(0, len(messages)):
  text=re.sub("[^a-zA-Z]"," ",messages["title"][i])
  text=text.lower()
  text=text.split()

# text is having list of words
# words which are not present in stopwords apply stemming
  text=[ps.stem(word) for word in text if not word in stopwords.words("english")]
  text=" ".join(text)
  corpus.append(text)

KeyError: ignored

In [ ]:
corpus

In [148]:
### One hot encoding
one_hot_repr=[one_hot(words,voc_size) for words in corpus]
one_hot_repr

[[3029, 3424, 4394, 1753, 4331, 2171, 3937, 1660, 898, 2907],
 [2901, 980, 1693, 3505, 4009, 113, 1767],
 [3741, 63, 3134, 4541],
 [2609, 772, 772, 934, 1074, 1505],
 [1605, 4009, 2519, 3101, 2187, 907, 4009, 2870, 2724, 3026],
 [4965,
  4009,
  4023,
  2360,
  4587,
  2717,
  1076,
  3338,
  2192,
  4129,
  787,
  1327,
  2599,
  210,
  1767]]

In [149]:
#corpus[1]
#one_hot_repr[1]

Padding

In [150]:
#### Padding
sent_length=20
embedded_docs= pad_sequences(one_hot_repr, padding="pre", maxlen=sent_length)
embedded_docs

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3029,
        3424, 4394, 1753, 4331, 2171, 3937, 1660,  898, 2907],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 2901,  980, 1693, 3505, 4009,  113, 1767],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 3741,   63, 3134, 4541],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 2609,  772,  772,  934, 1074, 1505],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1605,
        4009, 2519, 3101, 2187,  907, 4009, 2870, 2724, 3026],
       [   0,    0,    0,    0,    0, 4965, 4009, 4023, 2360, 4587, 2717,
        1076, 3338, 2192, 4129,  787, 1327, 2599,  210, 1767]],
      dtype=int32)

In [152]:
### Creating Model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])


In [153]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [154]:
len(embedded_docs), y.shape

(6, (18285,))

In [155]:
### convert into array
X_final= np.array(embedded_docs)
y_final=np.array(y)

In [156]:
X_final.shape , y_final.shape

((6, 20), (18285,))

In [157]:
## splitting
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test= train_test_split(X_final, y_final, test_size=0.25, random_state=42)


ValueError: ignored

Model Training

In [158]:
#### Training
model.fit(X_train , y_train, validation_data=(X_test,y_test), epochs=10, batch_size=64)

NameError: ignored

Performance metrics

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
## threshold value 0.5
y_pred = np.where(y_pred > 0.5, 1.0)

In [ ]:
from sklearn.metrics  import confusion_matrix
confusion_matrix(y_test,y_pred)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.metrics import classification_report
classification_report(y_test,y_pred)